#  Notebook Data Pipelines et Predict

**4 classes:**
- Data Handler
- Feature Recipe
- Future Extactor
- Model Builder

Data:
- https://storage.googleapis.com/h3-data/listings_final.csv
- https://storage.googleapis.com/h3-data/price_availability.csv

-------

In [1]:
#importer vos librairies  
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import sklearn as skn;

In [2]:
from sklearn.model_selection import train_test_split

### 0) Toutes les Classes

In [ ]:
import pandas as pd
import numpy as np
class DataHandler:
    """
        Getting data from bucket
    """
    def __init__(self):
        """
            Initialising the 3 datasets :
            entry 1
            entry 2
            result
        """
        print("DataHandlerintialisation")
        self.df_lf = None
        self.df_pa = None
        self.df_res = None
        print("intialisation done")

    def get_data(self):
        print("loading data from bucket")
        self.df_lf = pd.read_csv("https://storage.googleapis.com/h3-data/listings_final.csv",sep=';')
        self.df_pa = pd.read_csv("https://storage.googleapis.com/h3-data/price_availability.csv",sep=';')
        print("data loaded from bucket")

    def group_data(self):
        print("merging data")
        self.df_res = pd.merge(self.df_lf,self.df_pa.groupby('listing_id')['local_price'].mean('local_price'),how='inner', on='listing_id')
        print("size of the merged data : {} lines, {} columns".format(self.df_res.shape[0],self.df_res.shape[1]))

    def get_process_data(self):
        self.get_data()
        self.group_data()
        print("end of processing\n")

class FeatureRecipe:

    def __init__(self,df:pd.DataFrame):
        print("FeatureRecipe intialisation")
        self.df = df
        self.cate = []
        self.floa = []
        self.intt = []
        self.drop = []
        print("end of intialisation")

    def separate_variable_types(self) -> None:
        print("separating columns")
        for col in self.df.columns:
            if self.df[col].dtypes == int:
                self.intt.append(self.df[col])
            elif self.df[col].dtypes == float:
                self.floa.append(self.df[col])
            else:
                self.cate.append(self.df[col])
        print ("dataset column size : {} \nnumber of discreet values : {} \nnumber of continuous values : {} \nnumber of others : {} \ntaille total : {}".format(len(self.df.columns),len(self.intt),len(self.floa),len(self.cate),len(self.intt)+len(self.floa)+len(self.cate) ))

    def drop_na_prct(self,threshold : float):
        """
            on appelle la commande et on met un threshold entre 1 et 0 en flottant
            params: threshold : float
        """
        # par rapport a la colonne
        dropped = 0
        print("dropping columns with {} percentage ".format(threshold))
        for col in self.df.columns:
            if self.df[col].isna().sum()/self.df.shape[0] >= threshold:
                self.drop.append( self.df.drop([col], axis='columns', inplace=True) )
                dropped+=1
        print("dropped {} columns".format(dropped))

    def drop_useless_features(self):
        # droper les col vides et doublons de l'index et les colonnes qu'on va considerer inutile
        print("dropping useless columns")
        if 'Unnamed: 0' in self.df.columns:
            self.df.drop(['Unnamed: 0'], axis='columns', inplace=True)
        for col in self.df.columns:
            if self.df[col].isna().sum() == len(self.df):
                self.df.drop([col], axis='columns', inplace=True)
        print("done dropping")

    def drop_duplicate(self):
        # comparer les colonnes et voir si elles sont dupliquées
        print("dropping duplicated rows")
        self.df.drop_duplicates(inplace=True)
        duplicates = self.get_duplicates()
        for col in duplicates:
            self.df.drop(col)
        print("duplicated rows dropped")

    def get_duplicates(self):
        duplicates = []
        #for col in self.df.columns:
            #for scol in self.df.columns:
        for col in range(self.df.shape[1]-1):
            for scol in range(col+1,self.df.shape[1]-1):
                print("{} {}".format(col,scol))
                if sum( np.where(self.df.iloc[:,[col]] == self.df.iloc[:,[scol]],0,1) ) == 0:
                    duplicates.append(scol)
        return duplicates

    # def deal_date_time(self):
    #   pass

    def get_process_data(self,threshold : float):
        self.drop_useless_features()
        self.drop_na_prct(threshold)
        self.drop_duplicate()
        self.separate_variable_types()
        print("end of FeatureRecipe processing\n")

import sklearn as skn
import matplotlib as plt
from sklearn.model_selection import train_test_split

class FeatureExtractor:
    """
    Feature Extractor class
    """
    def __init__(self, data: pd.DataFrame, flist: list):
        """
            Input : pandas.DataFrame, feature list to drop
            Output : X_train, X_test, y_train, y_test according to sklearn.model_selection.train_test_split
        """
        self.X_train, self.X_test, self.y_train, self.y_test = None,None,None,None
        self.df = data
        self.flist = flist
    def splitting(size:float,rng:int,X pd.Series):
        self.X_train, self.X_test, self.y_train, self.y_test train_test_split(X, y, size, rng)

--------------

### 1) Classe DataHandler

- 3 attributs 

**V1**

In [ ]:
class DataHandler:
    """
        Get data from GSC Bucket 
    """
    def __init__(self):
        self.csvfile1 = None
        self.csvfile2 = None
        self.gouped_data = None
    def get_data(self):
        print(" - - - fetch data from gcs bucket : - - - ")
        listing = pd.read_csv('https://storage.googleapis.com/h3-data/listings_final.csv', sep=';')
        prices = pd.read_csv('https://storage.googleapis.com/h3-data/price_availability.csv', sep=';')
        self.csvfile1, self.csvfile2 = listing, prices  
        return " - - - data loaded - - - \nFiles : \n  - listing {} \n  - prices {}".format(listing.shape,prices.shape)
    def group_data(self):
        data = self.csvfile2.groupby('listing_id')['local_price'].mean()
        self.gouped_data = pd.merge(data, self.csvfile1, on='listing_id')
        print(" - - - data merged - - - ")
    def get_process_data(self):
        self.get_data()
        self.group_data()
        print(" - - - data processed - - - ")

**V2**

In [ ]:
class DataHandler:
    """
        Getting data from bucket
    """
    def __init__(self):
        """
            Initialising the 3 datasets :
            entry 1
            entry 2
            result
        """
        print("DataHandlerintialisation")
        self.df_lf = None
        self.df_pa = None
        self.df_res = None
        print("intialisation done")

    def get_data(self):
        print("loading data from bucket")
        self.df_lf = pd.read_csv("https://storage.googleapis.com/h3-data/listings_final.csv",sep=';')
        self.df_pa = pd.read_csv("https://storage.googleapis.com/h3-data/price_availability.csv",sep=';')
        print("data loaded from bucket")

    def group_data(self):
        print("merging data")
        self.df_res = pd.merge(self.df_lf,self.df_pa.groupby('listing_id')['local_price'].mean('local_price'),how='inner', on='listing_id')
        print("size of the merged data : {} lines, {} columns".format(self.df_res.shape[0],self.df_res.shape[1]))

    def get_process_data(self):
        self.get_data()
        self.group_data()
        print("end of processing\n")

In [6]:
d1 = DataHandler()

NameError: name 'DataHandler' is not defined

In [1]:
d1.get_process_data()
%time 

NameError: name 'df' is not defined

In [ ]:
d1.df_merge.head()

In [ ]:
d1.df_merge.tail()

In [ ]:
d1.df_merge.shape

----------------------

### 2) Classe FutureRecipe 

In [5]:
class FeatureRecipe:
    """
    Feature processing class
    """
    
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.continuous = None
        self.categorical = None
        self.discrete = None
        self.datetime = None
        
    #Separate features for feature engineering
    def separate_vraible_types(self) -> None:
    """ TODO : Diviser les types de variables dans un tableau """
        pass   
    
    #Useless feature
    def drop_uselessf(self):
    """ TODO : Supprimer les colonnes inutiles du dataset """
    #Soit celles en double soit celles avec NaN de 100%
        pass
    
    def deal_duplicate(self):
    """ TODO : Supprimer les lignes dupliquées du dataset """
        pass
    
    #NaN
    def drop_nanp(self, thresold: float):
    """ TODO : Supprimer un pourcentage de NA du dataset """
        """
        Drop NaN columns according to a thresold
        """
        def deal_nanp(df:pd.DataFrame, thresold: float):
            bf=[]
            for c in self.data.columns.to_list():
                if self.data[c].isna().sum()/self.data.shape[0] > thresold:
                    bf.append(c)
            logging.debug("{} feature have more than {} NaN ".format(len(bf), thresold))
            logging.debug('\n\n - - - features - - -  \n {}'.format(bf))
            return bf 
        self.data = self.data.drop(deal_nanp(self.data, thresold), axis=1)
        logging.debug('Some NaN features droped according to {} thresold'.format(thresold))
    
    #Datetime
    def deal_dtime(self):
    """ TODO : Traiter les DateTime """
        pass    
    #
    # extract data (optionnal)
    #
    
    def prepare_data(self, threshold: float):
        self.separate_variable_types()
        self.drop_uselessf()
        self.deal_duplicate()
        self.drop_nanp(threshold)
        self.deal_dtime()
     

IndentationError: expected an indented block (<ipython-input-5-322c2d7aa4a5>, line 15)

**V2**

In [ ]:
class FeatureRecipe:
    
    def __init__(self, data: pds.DataFrame):
        print("FeatureRecipe starts...")
        self.df = data
        self.cate = []
        self.floa = []
        self.intt = []
        print("End of FeatureRecipe initialisation\n")
    
    def separate_variable_types(self) -> None:
        print("Separate variable types starts...")
        for col in self.df.columns:
            if self.df[col].dtypes == int:
                self.intt.append(self.df[col])
            elif self.df[col].dtypes == float:
                self.floa.append(self.df[col])
            else:
                self.cate.append(self.df[col])
        print("Separate variable types end...")
        print ("Dataset number of columns : {} \nnumber of discreet values : {} \nnumber of continuous values : {} \nnumber of others : {} \ntotal size : {}\n".format(len(self.df.columns),
        len(self.intt),len(self.floa),len(self.cate),len(self.intt)+len(self.floa)+len(self.cate) ))
        
    def drop_uselessf(self):
        print("Drop useless feature start...")
        
        if "Unnamed: 0" in self.df.columns:
            self.df.drop("Unnamed: 0", axis=1, inplace=True)
            
        for col in self.df.columns:
            if self.df[col].isna().sum() == len(self.df[col]):
                self.df.drop([col], axis=1, inplace=True)
                
        print("Drop useless feature end...")
        print("Number columns remaining {}\n".format(len(self.df.columns)))
        
    def deal_duplicate(self):
        print("Deal duplicate start...")
        dropped_duplicates = self.df.drop_duplicates(inplace=True)
        print("Dropped duplicates : {}".format(dropped_duplicates))
        print("Deal duplicate end...")
    
    def drop_nanp(self, threshold: float):
        dropped = 0
        print("Drop columns with {} percentage of NAN".format(threshold))
        self.get_duplicates()
              
        for col in self.df.columns:
            if self.df[col].isna().sum() / self.df.shape[0] >= threshold:
                self.df.drop([col], axis=1, inplace=True)
                dropped+=1
              
        print("Number of columns dropped : {}\n".format(dropped))
    
    def get_duplicates(self):
        print("Get duplicates")
        
        for col in self.df.columns:
            for row in range(self.df.shape[0]):
                
              
    def deal_dtime(self):
        pass
    
    def prepare_data(self, threshold: float):
        self.drop_uselessf()
        self.separate_variable_types()
        self.deal_duplicate()
        self.drop_nanp(threshold)
        self.deal_dtime()

**Correction fonction Drop NaN**

In [ ]:
def drop_nanp(self, thresold: float):
        """
        Drop NaN columns according to a thresold
        """
        def deal_nanp(df:pd.DataFrame, thresold: float):
            bf=[]
            for c in self.data.columns.to_list():
                if self.data[c].isna().sum()/self.data.shape[0] > thresold:
                    bf.append(c)
            logging.debug("{} feature have more than {} NaN ".format(len(bf), thresold))
            logging.debug('\n\n - - - features - - -  \n {}'.format(bf))
            return bf 
        self.data = self.data.drop(deal_nanp(self.data, thresold), axis=1)
        logging.debug('Some NaN features droped according to {} thresold'.format(thresold))

In [ ]:
fterecipe = FeatureRecipe(d1.df_merge)

In [ ]:
fterecipe.prepare_data(3)

### 3) Classe FeatureExtractor 

In [ ]:
class FeatureExtractor:
    """
    Feature Extractor class
    """    
    def __init__(self, data: pd.DataFrame, flist: list):
        """
            Input : pandas.DataFrame, feature list to drop
            Output : X_train, X_test, y_train, y_test according to sklearn.model_selection.train_test_split
        """

**V2**

In [ ]:
class FeatureExtractor:
    """
        Feature Extractor class
    """
    
    def __init__(self, data : pds.DataFrame, flist: list):
        """
            Input : pandas.DataFrame, feature list to drop
            Output : X_train, X_test, y_train, y_test according to sklearn.model_selection.train_test_split
        """
        self.x_train, self.x_test, self.y_train, self.y_test = None, None, None, None
        self.df = data
        self.flist = flist
        
    def split(size: float, random_state :int = 42):
        pass
    
    def train():
        pass

In [ ]:
col.equals(otherCol)

**V3**

In [ ]:
class FeatureExtractor:
    """
    Feature Extractor class
    """
    def __init__(self, data: pd.DataFrame, flist: list):
        """
            Input : pandas.DataFrame, feature list to drop
            Output : X_train, X_test, y_train, y_test according to sklearn.model_selection.train_test_split
        """
        self.X_train, self.X_test, self.y_train, self.y_test = None,None,None,None
        self.df = data
        self.flist = flist
    def splitting(size:float,rng:int,X pd.Series):
        self.X_train, self.X_test, self.y_train, self.y_test train_test_split(X, y, size, rng)

class FeatureExtractor:
    """
    Feature Extractor class
    """
    def __init__(self, data: pd.DataFrame, flist: list):
        """
            Input : pandas.DataFrame, feature list to drop
            Output : X_train, X_test, y_train, y_test according to sklearn.model_selection.train_test_split
        """
        self.X_train, self.X_test, self.y_train, self.y_test = None,None,None,None
        self.df = data
        self.flist = flist
    def splitting(size:float,rng:int,X pd.Series):
        self.X_train, self.X_test, self.y_train, self.y_test train_test_split(X, y, size, rng)

### 4) Classe ModelBuilder

In [ ]:
class ModelBuilder:
    """
        Class for train and print results of ml model 
    """
    def __init__(self, model_path: str = None, save: bool = None):
        pass
    def __repr__(self):
        pass
    def predict(self, X) -> np.ndarray:
        pass
    def save_model(self, path:str):
        #with the format : 'model_{}_{}'.format(date)
        pass
    def load_model(self):
        try:
            #load model
            pass
        except:
            pass

In [ ]:
def DataManager(d:DataHandler=None, fr: FeatureRecipe=None, fe:FeatureExtractor=None):
    """
        Fonction qui lie les 4 classes de la pipeline et qui return FeatureExtractor.split(0.1)
    """
    pass
#on appelera la fonction DataManager() de la facon suivante : 
X_train, X_test, y_train, y_test = DataManager()

In [ ]:
m = ModelBuilder() 
m.train(X_train, y_train)
m.print_accuracy()
m.save_model(path)